In [3]:
import os
import requests
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.applications import VGG16
from tensorflow.keras import Model

from tensorflow.keras.applications import ResNet50

import joblib

2024-07-08 01:03:23.829577: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-08 01:03:24.686225: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
# URLs das imagens para download
image_urls ={
    "calcado_infantil":[
        "https://images-americanas.b2w.io/produtos/5017625269/imagens/mordedor-bebe-maozinha-com-agua-alivio-gengiva-buba-baby/5017625357_1_large.jpg"
    ]
    
}

In [ ]:
# Crie um diretório para armazenar as imagens baixadas
os.makedirs('images', exist_ok=True)

# Função para baixar imagens
def download_images(urls):
    for chave, lista in urls.items():
        print(f"Carregando as imagens da categoria {chave}")
        # Cria o diretório para a chave se ele não existir
        os.makedirs(f'images/{chave}', exist_ok=True)
        for i, url in enumerate(lista):
            response = requests.get(url)
            if response.status_code == 200:  # Verifica se o download foi bem-sucedido
                with open(f'images/{chave}/image_{i}.jpg', 'wb') as file:
                    file.write(response.content)
            else:
                print(f"Falha ao baixar a imagem {url}")


# Baixe as imagens
download_images(image_urls)

Carregando as imagens da categoria calcado_infantil


In [ ]:
def load_and_display_images(main_folder, num_images=2):
    for subdir in os.listdir(main_folder):
        subdir_path = os.path.join(main_folder, subdir)
        if os.path.isdir(subdir_path):
            print(f"Exibindo imagens da subpasta: {subdir}")
            image_files = os.listdir(subdir_path)
            for image_file in image_files[:num_images]:
                img_path = os.path.join(subdir_path, image_file)
                try:
                    img = Image.open(img_path)
                    plt.imshow(img)
                    plt.title(f'Categoria: {subdir}, Imagem: {image_file}')
                    plt.axis('off') 
                    plt.show()
                except Exception as e:
                    print(f"Erro ao carregar a imagem {img_path}: {e}")

# Carregue e visualize as imagens baixadas
load_and_display_images('images')

In [ ]:
def preprocess_images(main_folder, target_size=(224, 224)):
    processed_images = []
    labels = []
    for subdir in os.listdir(main_folder):
        subdir_path = os.path.join(main_folder, subdir)
        if os.path.isdir(subdir_path):
            for image_file in os.listdir(subdir_path):
                img_path = os.path.join(subdir_path, image_file)
                try:
                    img = Image.open(img_path).resize(target_size)
                    if img.mode != 'RGB':
                        img = img.convert('RGB') 
                    img_array = np.array(img) / 255.0 
                    if img_array.shape == (224, 224, 3): 
                        processed_images.append(img_array)
                        labels.append(subdir)  # Usa o nome do subdiretório como rótulo
                    else:
                        print(f"Imagem {img_path} tem forma incorreta: {img_array.shape}")
                except Exception as e:
                    print(f"Erro ao processar a imagem {img_path}: {e}")
    return np.array(processed_images), np.array(labels)

# Pré-processamento das imagens
processed_images, labels = preprocess_images('images')

# Exibir as formas dos arrays processados
print(f"Processed images shape: {processed_images.shape}")
print(f"Labels shape: {labels.shape}")

# Codificação dos rótulos para números inteiros
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Número de classes
num_classes = len(np.unique(encoded_labels))
print(f"Number of classes: {num_classes}")

In [ ]:
# Divisão dos dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(processed_images, encoded_labels, test_size=0.2, random_state=42)

# Definição do modelo CNN
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

######
# Carregar o modelo ResNet50 pré-treinado
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Adicionar camadas ao modelo
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Congelar as camadas convolucionais do modelo base
base_model.trainable = False
###################

# Compilação do modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Treinamento do modelo
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

# Avaliação do modelo
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')

# Previsão com o modelo treinado
predictions = model.predict(X_test)

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Adicionar novas camadas de classificação
x = base_model.output
x = layers.Flatten()(x)
x = layers.Dense(1024, activation='relu')(x)
predictions = layers.Dense(num_classes, activation='softmax')(x)

# Criar o novo modelo
model = Model(inputs=base_model.input, outputs=predictions)

# Congelar as camadas do modelo base
for layer in base_model.layers:
    layer.trainable = False

# Compilação do modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Treinamento do modelo com Transfer Learning
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

# Avaliação do modelo
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')

In [9]:
from io import BytesIO

# Função para carregar e pré-processar a nova imagem
def load_and_preprocess_image(image_url, target_size=(224, 224)):
    response = requests.get(image_url)
    img = Image.open(BytesIO(response.content)).resize(target_size)
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img_array = np.array(img) / 255.0  # Normalize para [0, 1]
    return np.expand_dims(img_array, axis=0)  # Adiciona uma dimensão extra para batch

# Função para prever a classe de uma nova imagem
def predict_image_category(image_url):
    img_array = load_and_preprocess_image(image_url)
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)[0]
    predicted_label = label_encoder.inverse_transform([predicted_class])[0]
    print(predicted_label)
    probabilities = prediction[0]
    # Converter as probabilidades em porcentagens
    probabilities_percent = np.max(prediction) * 100
    return predicted_label, probabilities_percent

# URL da nova imagem para classificação
new_image_url = "https://photos.enjoei.com.br/cadeira-alimentacao-feed-safety-first-estofado-novo-97365579/800x800/czM6Ly9waG90b3MuZW5qb2VpLmNvbS5ici9wcm9kdWN0cy81MjczNTc5LzYzMTgwMGY1MWQxYzM2MzQ0N2EwZDlkZDg4YWU5YTQ5LmpwZw"

# Classificação da nova imagem
predicted_category, probabilities = predict_image_category(new_image_url)
print(f'A nova imagem pertence à categoria: {predicted_category}')
print(f'Probabilidades de cada classe: {probabilities}%')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step


AttributeError: 'numpy.ndarray' object has no attribute 'inverse_transform'

In [ ]:
model.save("./model1.keras")

In [2]:
model = tf.keras.models.load_model("./model1.keras")

2024-06-27 10:00:30.765068: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2024-06-27 10:00:30.765089: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:134] retrieving CUDA diagnostic information for host: eduardo-ubuntu
2024-06-27 10:00:30.765094: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:141] hostname: eduardo-ubuntu
2024-06-27 10:00:30.765164: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:165] libcuda reported version is: 535.183.1
2024-06-27 10:00:30.765178: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:169] kernel reported version is: 535.171.4
2024-06-27 10:00:30.765182: E external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:251] kernel version 535.171.4 does not match DSO version 535.183.1 -- cannot find working devices in this configuration


In [6]:
label_encoder = joblib.load("modelo_treinado_joblib.sav")
print(label_encoder.in)
num_classes = len(np.unique(encoded_labels))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5]


In [ ]:
arquivo_joblib = "modelo_treinado_joblib.sav"
joblib.dump(encoded_labels, arquivo_joblib)